<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [34]:
import csv

from collections import namedtuple   # Convenient to store the data rows

import pandas as pd

DATA_FILE = './data/chipotle.tsv'

In [35]:
file_nested_list = pd.read_csv(DATA_FILE, delimiter='\t')

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [36]:
data_headers = file_nested_list.columns

In [37]:
data = pd.read_csv(DATA_FILE, delimiter='\t',header=None,skiprows=1)
data.head(1)

,0,1,2,3,4
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39


---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [49]:
file_nested_list.head(20)

,order_id,quantity,item_name,choice_description,item_price,total_item_cost
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39
1,1,1,Izze,[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,33.96
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,10.98
6,3,1,Side of Chips,NaN,1.69,1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75,11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25,9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25,9.25


In [39]:
file_nested_list.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

In [40]:
#convert 'item_price' to an int in order to use groupby / .sum()
file_nested_list['item_price'] = file_nested_list['item_price'].replace('[\$,]', '', regex=True).astype(float)

In [48]:
file_nested_list.groupby('order_id')['item_price'].sum().mean()

18.81142857142869

In [42]:
#quantity is important, will add column that is price * quantity and sum() by that to get total price of each order

In [44]:
file_nested_list["total_item_cost"] = file_nested_list["quantity"] * file_nested_list["item_price"]

In [45]:
file_nested_list.head(1)

,order_id,quantity,item_name,choice_description,item_price,total_item_cost
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39


In [47]:
file_nested_list.groupby('order_id')['total_item_cost'].sum().mean()

21.394231188658654

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [65]:
soda_mask = file_nested_list["item_name"].isin(['Canned Soda','Canned Soft Drink'])
soda_df = file_nested_list.loc[soda_mask,:]
sodas = set(soda_df["choice_description"])
sodas

{'[Coca Cola]',
 '[Coke]',
 '[Diet Coke]',
 '[Diet Dr. Pepper]',
 '[Dr. Pepper]',
 '[Lemonade]',
 '[Mountain Dew]',
 '[Nestea]',
 '[Sprite]'}

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
